In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

In [2]:
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel
import torch
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset

In [3]:
# Load the SWDA dataset
swda = load_dataset("swda")

swda

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/213543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56729 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4514 [00:00<?, ? examples/s]

Dataset swda downloaded and prepared to /root/.cache/huggingface/datasets/swda/default/0.0.0/b53d17ec4c6e31d0921591dd2d8e86d15850822209a980fcddb2983fc948e499. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'text', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_education', 'to_caller_birth_year', 'to_caller_dialect_area'],
        num_rows: 213543
    })
    validation: Dataset({
        features: ['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'text', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_educ

In [4]:
# Define the data preprocessing function
def preprocess_data(utterance):
    text = utterance['text']
    label = utterance['damsl_act_tag']
    return {'text': text, 'label': label}
# Preprocess the data
swda = swda.map(preprocess_data)

  0%|          | 0/213543 [00:00<?, ?ex/s]

  0%|          | 0/56729 [00:00<?, ?ex/s]

  0%|          | 0/4514 [00:00<?, ?ex/s]

In [5]:
swda = swda.remove_columns(['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_education', 'to_caller_birth_year', 'to_caller_dialect_area'])

In [6]:
swda

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 213543
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 56729
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4514
    })
})

In [7]:
import pandas as pd
sorted(pd.DataFrame(swda["train"])["label"].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42]

In [8]:
# Tokenize the input and context
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
swda = swda.map(lambda x: tokenizer(x['text'], padding='max_length', truncation=True), batched=True)

  0%|          | 0/214 [00:00<?, ?ba/s]

  0%|          | 0/57 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [9]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset


class SWDADataset(Dataset):
    def __init__(self, swda, tokenizer, max_seq_length):
        self.swda = swda
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.swda)

    def __getitem__(self, index):
        # Get input text
        input_text = self.tokenizer.encode_plus(
            self.swda[index]['text'],
            add_special_tokens=True,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Get context
        context = self.swda[index-1] if index > 0 else self.swda[index]
        context = self.tokenizer.encode_plus(
            context['text'],
            add_special_tokens=True,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Get label
        label = self.swda[index]['label']
        
        return input_text, context, label


# Create the train loader
train_dataset = SWDADataset(swda['train'], tokenizer, max_seq_length=128)
val_dataset = SWDADataset(swda['validation'], tokenizer, max_seq_length=128)
test_dataset = SWDADataset(swda['test'], tokenizer, max_seq_length=128)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [10]:
train_dataset[0]

({'input_ids': tensor([[ 101, 3100, 1012, 1013,  102,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# Define the model architecture
# Define model architecture
class RNN(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(RNN, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        for param in self.bert.parameters():
            param.requires_grad = False
        self.rnn = nn.GRU(input_size=768, hidden_size=hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, input_text, context):
        input_ids = input_text['input_ids'].to(device)
        attention_mask = input_text['attention_mask'].to(device)
        context_ids = context['input_ids'].to(device)
        context_mask = context['attention_mask'].to(device)
        
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled_output = outputs.pooler_output

        context_outputs = self.bert(input_ids=context_ids, attention_mask=context_mask, return_dict=True)
        pooled_context_output = context_outputs.pooler_output
        
        
        output, _ = self.rnn(pooled_output.unsqueeze(1))
        context_output, _ = self.rnn(pooled_context_output.unsqueeze(1))
        output = self.fc1(output)
        output = torch.cat((output.squeeze(1), context_output.squeeze(1)), dim=1)
        output = self.fc2(output)
        return output

# Instantiate the model
# Instantiate model and move it to device
model = RNN(hidden_size=256, num_classes=43)
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [14]:
import os
#os.environ["CUDA_LAUNCH_BLOCKING"]= "true"

In [15]:
num_epochs = 5

best_val_loss = float('inf')
for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for i, (input_text, context, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        input_text = {
            'input_ids': input_text['input_ids'].squeeze(1),
            'attention_mask': input_text['attention_mask'].squeeze(1),
            'token_type_ids': input_text['token_type_ids'].squeeze(1)
        }
        context = {
            'input_ids': context['input_ids'].squeeze(1),
            'attention_mask': context['attention_mask'].squeeze(1),
            'token_type_ids': context['token_type_ids'].squeeze(1)
        }
        labels = labels.to(device)
        output = model(input_text, context)
        loss = criterion(output, labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * input_text["input_ids"].size(0)

        # Compute accuracy for this batch
        predicted_labels = output.argmax(dim=1)
        num_correct = (predicted_labels == labels).sum().item()
        acc = num_correct / len(labels)
        epoch_acc += acc
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Train Loss: {loss.item():.4f}')

    # Compute average loss and accuracy for epoch
    epoch_loss /= len(train_loader)
    epoch_acc /= len(train_loader)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
    
    # Validation loop
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for input_text, context, labels in val_loader:
            input_text = {
                'input_ids': input_text['input_ids'].squeeze(1),
                'attention_mask': input_text['attention_mask'].squeeze(1),
                'token_type_ids': input_text['token_type_ids'].squeeze(1)
            }
            context = {
                'input_ids': context['input_ids'].squeeze(1),
                'attention_mask': context['attention_mask'].squeeze(1),
                'token_type_ids': context['token_type_ids'].squeeze(1)
            }
            labels = labels.to(device)
            output = model(input_text, context)
            loss = criterion(output, labels)

            val_loss += loss.item() * input_text["input_ids"].size(0)

            # Compute accuracy for this batch
            predicted_labels = output.argmax(dim=1)
            num_correct = (predicted_labels == labels).sum().item()
            acc = num_correct / len(labels)
            val_acc += acc

    # Compute average validation loss and accuracy
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pt')


Epoch [1/5], Step [100/1669], Train Loss: 1.8914
Epoch [1/5], Step [200/1669], Train Loss: 1.8300
Epoch [1/5], Step [300/1669], Train Loss: 1.5354
Epoch [1/5], Step [400/1669], Train Loss: 1.4443
Epoch [1/5], Step [500/1669], Train Loss: 1.5466
Epoch [1/5], Step [600/1669], Train Loss: 1.3846
Epoch [1/5], Step [700/1669], Train Loss: 1.2945
Epoch [1/5], Step [800/1669], Train Loss: 1.3506
Epoch [1/5], Step [900/1669], Train Loss: 1.5413
Epoch [1/5], Step [1000/1669], Train Loss: 1.2017
Epoch [1/5], Step [1100/1669], Train Loss: 1.1159
Epoch [1/5], Step [1200/1669], Train Loss: 1.3620
Epoch [1/5], Step [1300/1669], Train Loss: 1.1618
Epoch [1/5], Step [1400/1669], Train Loss: 1.1463
Epoch [1/5], Step [1500/1669], Train Loss: 1.2350
Epoch [1/5], Step [1600/1669], Train Loss: 1.1519
Epoch 1/5, Loss: 176.2643, Accuracy: 0.5839
Epoch 1/5, Val Loss: 75.4238, Val Accuracy: 0.6367
Epoch [2/5], Step [100/1669], Train Loss: 1.0988
Epoch [2/5], Step [200/1669], Train Loss: 1.0906
Epoch [2/5], Ste

In [16]:
model.eval()
test_loss = 0
test_acc = 0
with torch.no_grad():
    for input_text, context, labels in test_loader:
        input_text = {
            'input_ids': input_text['input_ids'].squeeze(1),
            'attention_mask': input_text['attention_mask'].squeeze(1),
            'token_type_ids': input_text['token_type_ids'].squeeze(1)
        }
        context = {
            'input_ids': context['input_ids'].squeeze(1),
            'attention_mask': context['attention_mask'].squeeze(1),
            'token_type_ids': context['token_type_ids'].squeeze(1)
        }
        labels = labels.to(device)
        output = model(input_text, context)
        loss = criterion(output, labels)

        test_loss += loss.item() * input_text["input_ids"].size(0)

        # Compute accuracy for this batch
        predicted_labels = output.argmax(dim=1)
        num_correct = (predicted_labels == labels).sum().item()
        acc = num_correct / len(labels)
        test_acc += acc

# Compute average test loss and accuracy
test_loss /= len(test_loader)
test_acc /= len(test_loader)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')


Test Loss: 66.5564, Test Accuracy: 0.6716
